In [1]:
import tensorflow as tf
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.casual import TweetTokenizer
from w266_common import utils, vocabulary
import time
import datetime
import os


/home/saulgrimaldo1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import csv
tokenizer = TweetTokenizer()
x_data = []
x_contexts = []
labels = []
sentences  = []
contexts = []
with open('merged_data_v3.csv', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            continue
        sentence, context, sarcasm = row
        sentences.append(sentence)
        contexts.append(context)
        x_tokens = utils.canonicalize_words(tokenizer.tokenize(sentence), hashtags = True)
        context_tokens = utils.canonicalize_words(tokenizer.tokenize(context), hashtags = True)
        x_data.append(x_tokens)
        x_contexts.append(context_tokens)
        labels.append(int(sarcasm))


#rng = np.random.RandomState(5)
#rng.shuffle(x_data)  # in-place
#train_split_idx = int(0.7 * len(labels))
#test_split_idx  = int(0.9 * len(labels))

shuffle_indices = np.random.permutation(np.arange(len(labels)))
train_split_idx = int(0.7 * len(labels))
test_split_idx  = int(0.9 * len(labels))

train_indices = shuffle_indices[:train_split_idx]
validation_indices = shuffle_indices[train_split_idx:test_split_idx]
test_indices = shuffle_indices[test_split_idx:]


train_sentences = np.array(x_data)[train_indices]
train_contexts = np.array(x_contexts)[train_indices]
train_labels= np.array(labels)[train_indices] 
validation_sentences = np.array(x_data)[validation_indices]
validation_labels = np.array(labels)[validation_indices]
validation_contexts = train_contexts = np.array(x_contexts)[validation_indices]
test_sentences = np.array(x_data)[test_indices]  
test_contexts = train_contexts = np.array(x_contexts)[test_indices]
test_labels = np.array(labels)[test_indices]  

In [3]:
a = [2]*4
a[2] = 6
a

[2, 2, 6, 2]

In [4]:
def transform_labels(raw_label_set, size):
    label_set = []
    for label in raw_label_set:
        labels = [0] * size
        labels[label] = 1
        label_set.append(labels)
    return np.array(label_set)

expanded_train_labels = transform_labels(train_labels, 2)
expanded_validation_labels = transform_labels(validation_labels,2)
expanded_test_labels = transform_labels(test_labels,2)

In [5]:
a = [1,3,4,5,6,7,8,8,1,5,6,7]
a + ["<PADDING>"]*0

[1, 3, 4, 5, 6, 7, 8, 8, 1, 5, 6, 7]

In [6]:
class PaddingAndTruncating:
    def __init__(self, max_len):
        self.max_len = max_len
    
    def pad_or_truncate(self, sentence):
        sen_len = len(sentence)
        paddings = self.max_len - sen_len
        if paddings >=0:
            return sentence + ["<PADDING>"] * paddings
        return sentence[0:paddings]
        
PadAndTrunc = PaddingAndTruncating(10)
        
        
PadAndTrunc.pad_or_truncate(["the","angry","man","is","angry"])

['the',
 'angry',
 'man',
 'is',
 'angry',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [7]:
from tensorflow.contrib import learn

#max_len = max([len(sent) for sent  in train_sentences])
PadAndTrunc =PaddingAndTruncating(30)
train_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, train_sentences))
train_context_padded = list(map(PadAndTrunc.pad_or_truncate, train_contexts))
validation_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, validation_sentences))
validation_context_padded = list(map(PadAndTrunc.pad_or_truncate, validation_contexts))
test_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, test_sentences))
test_context_padded = list(map(PadAndTrunc.pad_or_truncate, test_contexts))

vocab = vocabulary.Vocabulary(utils.flatten(list(train_sentences_padded) + list(train_context_padded)), 2500)
train_s = np.array(list(map(vocab.words_to_ids, train_sentences_padded)))
train_c = np.array(list(map(vocab.words_to_ids, train_context_padded)))
validation_s = np.array(list(map(vocab.words_to_ids, validation_sentences_padded)))
validation_c = np.array(list(map(vocab.words_to_ids, validation_context_padded)))
test_s = np.array(list(map(vocab.words_to_ids, test_sentences_padded)))
test_c = np.array(list(map(vocab.words_to_ids, test_context_padded)))
train_s

array([[   2, 1040,  267, ...,    3,    3,    3],
       [ 947,  275,  947, ...,    4,    3,    3],
       [2203,   20,    5, ...,    4,    4,    4],
       ..., 
       [  10,    5,    7, ...,    3,    3,    3],
       [  70,   19,   44, ...,   24,    2,    2],
       [  10,    5,    7, ...,    3,    3,    3]])

In [8]:
i = 3
("conv-maxpool-%s" % i)

'conv-maxpool-3'

In [9]:
class TextCNN(object):

    def __init__(
      self, sequence_length, num_classes, vocab_size, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + avgpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-avgpool-%s" % i) as scope:
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]

                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
              
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded1,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h1 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh1")
               
                # Maxpooling over the outputs
                pooled = tf.nn.avg_pool(
                    h1,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
                pooled_outputs.append(pooled)
                #pooled_outputs.append(pooled)
                scope.reuse_variables()
                 
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded2,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h2 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh2")
                # Maxpooling over the outputs
                pooled2 = tf.nn.avg_pool(
                    h2,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled2)

        # Combine all the pooled features

        num_filters_total = num_filters * len(filter_sizes) * 2

        self.h_pool = tf.concat(pooled_outputs, 1)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
        
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            self.labels = tf.argmax(self.input_y, 1)
            TP = tf.count_nonzero(self.predictions * self.labels)
            TN = tf.count_nonzero((self.predictions - 1) * (self.labels - 1))
            FP = tf.count_nonzero(self.predictions * (self.labels - 1))
            FN = tf.count_nonzero((self.predictions - 1) * self.labels)
            correct_predictions = tf.equal(self.predictions, self.labels)
            self.precision = TP / (TP + FP)
            self.recall = TP / (TP + FN)
            self.f1_score = 2 * self.precision * self.recall / (self.precision + self.recall)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [10]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [11]:
learning_rate = 0.1

tf.flags.DEFINE_float("dev_sample_percentage", 0.75, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 50, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 50, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.4, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size = 100
tf.flags.DEFINE_integer("batch_size", batch_size, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():

        cnn = TextCNN(
            sequence_length=train_s.shape[1],
            num_classes=2,
            vocab_size=2500,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        
         # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdagradOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch1, x_batch2, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch1, x_batch2, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1 = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(train_s, train_c,  expanded_train_labels)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch1, x_batch2, y_batch = zip(*batch)
            train_step(x_batch1, x_batch1, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(validation_s, validation_c, expanded_validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
print("\nTest Set:")
dev_step(test_s, test_c, expanded_test_labels, writer=dev_summary_writer)
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=100
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.75
DROPOUT_KEEP_PROB=0.4
EMBEDDING_DIM=50
EVALUATE_EVERY=20
FILTER_SIZES=1
L2_REG_LAMBDA=0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=50
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/hist is illegal; using conv-avgpool-0/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/sparsity is illegal; using conv-avgpool-0/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/hist is illegal; using conv-avgpool-0/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad

2018-04-17T05:49:32.843699: step 103, loss 0.55133, acc 0.72
2018-04-17T05:49:32.865051: step 104, loss 0.512383, acc 0.76
2018-04-17T05:49:32.886173: step 105, loss 0.502151, acc 0.8
2018-04-17T05:49:32.907644: step 106, loss 0.582191, acc 0.74
2018-04-17T05:49:32.929923: step 107, loss 0.475634, acc 0.83
2018-04-17T05:49:32.947547: step 108, loss 0.463896, acc 0.780822
2018-04-17T05:49:32.969393: step 109, loss 0.520532, acc 0.78
2018-04-17T05:49:32.990923: step 110, loss 0.510371, acc 0.86
2018-04-17T05:49:33.012590: step 111, loss 0.464261, acc 0.8
2018-04-17T05:49:33.033863: step 112, loss 0.588559, acc 0.74
2018-04-17T05:49:33.059580: step 113, loss 0.53799, acc 0.68
2018-04-17T05:49:33.081081: step 114, loss 0.522884, acc 0.74
2018-04-17T05:49:33.103291: step 115, loss 0.599659, acc 0.73
2018-04-17T05:49:33.125080: step 116, loss 0.561278, acc 0.73
2018-04-17T05:49:33.142735: step 117, loss 0.511252, acc 0.739726
2018-04-17T05:49:33.164267: step 118, loss 0.526402, acc 0.79
2018

2018-04-17T05:49:36.141896: step 223, loss 0.488231, acc 0.72
2018-04-17T05:49:36.163023: step 224, loss 0.469266, acc 0.74
2018-04-17T05:49:36.180093: step 225, loss 0.486819, acc 0.808219
2018-04-17T05:49:36.202113: step 226, loss 0.518929, acc 0.76
2018-04-17T05:49:36.223664: step 227, loss 0.380652, acc 0.86
2018-04-17T05:49:36.246616: step 228, loss 0.537248, acc 0.76
2018-04-17T05:49:36.267660: step 229, loss 0.398349, acc 0.86
2018-04-17T05:49:36.288729: step 230, loss 0.486653, acc 0.76
2018-04-17T05:49:36.310541: step 231, loss 0.53663, acc 0.75
2018-04-17T05:49:36.332306: step 232, loss 0.427031, acc 0.78
2018-04-17T05:49:36.358095: step 233, loss 0.440561, acc 0.8
2018-04-17T05:49:36.375703: step 234, loss 0.458256, acc 0.780822
2018-04-17T05:49:36.397834: step 235, loss 0.501407, acc 0.79
2018-04-17T05:49:36.418761: step 236, loss 0.458079, acc 0.82
2018-04-17T05:49:36.439667: step 237, loss 0.488686, acc 0.79
2018-04-17T05:49:36.460584: step 238, loss 0.704387, acc 0.62
20

2018-04-17T05:49:39.396931: step 345, loss 0.443701, acc 0.8
2018-04-17T05:49:39.418289: step 346, loss 0.474488, acc 0.8
2018-04-17T05:49:39.439309: step 347, loss 0.443613, acc 0.8
2018-04-17T05:49:39.459973: step 348, loss 0.427242, acc 0.77
2018-04-17T05:49:39.481532: step 349, loss 0.456031, acc 0.78
2018-04-17T05:49:39.502286: step 350, loss 0.387956, acc 0.84
2018-04-17T05:49:39.519163: step 351, loss 0.354696, acc 0.863014
2018-04-17T05:49:39.540423: step 352, loss 0.449158, acc 0.81
2018-04-17T05:49:39.564158: step 353, loss 0.405711, acc 0.77
2018-04-17T05:49:39.585011: step 354, loss 0.394432, acc 0.85
2018-04-17T05:49:39.612226: step 355, loss 0.416413, acc 0.83
2018-04-17T05:49:39.633879: step 356, loss 0.504906, acc 0.75
2018-04-17T05:49:39.655090: step 357, loss 0.366683, acc 0.83
2018-04-17T05:49:39.676386: step 358, loss 0.338311, acc 0.83
2018-04-17T05:49:39.698050: step 359, loss 0.478288, acc 0.8
2018-04-17T05:49:39.715001: step 360, loss 0.483864, acc 0.753425

Eva

2018-04-17T05:49:42.604275: step 465, loss 0.375132, acc 0.87
2018-04-17T05:49:42.624683: step 466, loss 0.34823, acc 0.84
2018-04-17T05:49:42.647951: step 467, loss 0.329652, acc 0.84
2018-04-17T05:49:42.665112: step 468, loss 0.323308, acc 0.835616
2018-04-17T05:49:42.686547: step 469, loss 0.333851, acc 0.87
2018-04-17T05:49:42.707922: step 470, loss 0.343726, acc 0.85
2018-04-17T05:49:42.729168: step 471, loss 0.454869, acc 0.82
2018-04-17T05:49:42.750262: step 472, loss 0.374604, acc 0.85
2018-04-17T05:49:42.771867: step 473, loss 0.418089, acc 0.8
2018-04-17T05:49:42.792736: step 474, loss 0.450577, acc 0.81
2018-04-17T05:49:42.817700: step 475, loss 0.389007, acc 0.79
2018-04-17T05:49:42.839185: step 476, loss 0.404742, acc 0.82
2018-04-17T05:49:42.856116: step 477, loss 0.391134, acc 0.835616
2018-04-17T05:49:42.878698: step 478, loss 0.343, acc 0.86
2018-04-17T05:49:42.905893: step 479, loss 0.422796, acc 0.8
2018-04-17T05:49:42.927448: step 480, loss 0.359596, acc 0.79

Evalu

...

2018-04-17T05:50:06.168946: step 1312, loss 0.202433, acc 0.9
2018-04-17T05:50:06.192263: step 1313, loss 0.175878, acc 0.94
2018-04-17T05:50:06.208968: step 1314, loss 0.14303, acc 0.972603
2018-04-17T05:50:06.230457: step 1315, loss 0.17476, acc 0.94
2018-04-17T05:50:06.251317: step 1316, loss 0.205197, acc 0.91
2018-04-17T05:50:06.272107: step 1317, loss 0.18178, acc 0.94
2018-04-17T05:50:06.293644: step 1318, loss 0.194595, acc 0.96
2018-04-17T05:50:06.316006: step 1319, loss 0.199892, acc 0.91
2018-04-17T05:50:06.340439: step 1320, loss 0.286794, acc 0.91

Evaluation:
2018-04-17T05:50:06.445512: step 1320, loss 0.596024, acc 0.72149, rec 0.927024, pre 0.6583, f1 0.769886

2018-04-17T05:50:06.470331: step 1321, loss 0.205829, acc 0.91
2018-04-17T05:50:06.496126: step 1322, loss 0.216745, acc 0.93
2018-04-17T05:50:06.517528: step 1323, loss 0.211068, acc 0.917808
2018-04-17T05:50:06.541607: step 1324, loss 0.185689, acc 0.94
2018-04-17T05:50:06.563399: step 1325, loss 0.202652, acc 

2018-04-17T05:50:09.569155: step 1431, loss 0.238971, acc 0.876712
2018-04-17T05:50:09.591232: step 1432, loss 0.184447, acc 0.95
2018-04-17T05:50:09.613199: step 1433, loss 0.223413, acc 0.88
2018-04-17T05:50:09.634770: step 1434, loss 0.197351, acc 0.93
2018-04-17T05:50:09.656539: step 1435, loss 0.12728, acc 0.97
2018-04-17T05:50:09.677802: step 1436, loss 0.202283, acc 0.9
2018-04-17T05:50:09.699411: step 1437, loss 0.242934, acc 0.91
2018-04-17T05:50:09.722951: step 1438, loss 0.236221, acc 0.9
2018-04-17T05:50:09.744990: step 1439, loss 0.138781, acc 0.96
2018-04-17T05:50:09.761961: step 1440, loss 0.27208, acc 0.90411

Evaluation:
2018-04-17T05:50:09.868980: step 1440, loss 0.515018, acc 0.77192, rec 0.63626, pre 0.875981, f1 0.73712

2018-04-17T05:50:09.893826: step 1441, loss 0.298488, acc 0.88
2018-04-17T05:50:09.917818: step 1442, loss 0.190196, acc 0.94
2018-04-17T05:50:09.939743: step 1443, loss 0.294967, acc 0.87
2018-04-17T05:50:09.961121: step 1444, loss 0.158625, acc 0

2018-04-17T05:50:13.174175: step 1549, loss 0.19604, acc 0.91
2018-04-17T05:50:13.199008: step 1550, loss 0.141411, acc 0.93
2018-04-17T05:50:13.220874: step 1551, loss 0.196886, acc 0.93
2018-04-17T05:50:13.242261: step 1552, loss 0.204375, acc 0.91
2018-04-17T05:50:13.264102: step 1553, loss 0.133804, acc 0.95
2018-04-17T05:50:13.285087: step 1554, loss 0.247748, acc 0.94
2018-04-17T05:50:13.306662: step 1555, loss 0.210772, acc 0.93
2018-04-17T05:50:13.327705: step 1556, loss 0.169991, acc 0.92
2018-04-17T05:50:13.345413: step 1557, loss 0.150141, acc 0.945205
2018-04-17T05:50:13.367247: step 1558, loss 0.203011, acc 0.9
2018-04-17T05:50:13.388401: step 1559, loss 0.158491, acc 0.95
2018-04-17T05:50:13.414950: step 1560, loss 0.150619, acc 0.95

Evaluation:
2018-04-17T05:50:13.519108: step 1560, loss 0.830717, acc 0.640688, rec 0.95553, pre 0.587658, f1 0.727746

2018-04-17T05:50:13.542612: step 1561, loss 0.226402, acc 0.92
2018-04-17T05:50:13.564280: step 1562, loss 0.167374, acc 

2018-04-17T05:50:16.518840: step 1670, loss 0.144749, acc 0.94
2018-04-17T05:50:16.540047: step 1671, loss 0.180643, acc 0.94
2018-04-17T05:50:16.561203: step 1672, loss 0.176647, acc 0.92
2018-04-17T05:50:16.582256: step 1673, loss 0.161121, acc 0.91
2018-04-17T05:50:16.598880: step 1674, loss 0.142885, acc 0.958904
2018-04-17T05:50:16.620667: step 1675, loss 0.116732, acc 0.95
2018-04-17T05:50:16.641784: step 1676, loss 0.140938, acc 0.97
2018-04-17T05:50:16.662864: step 1677, loss 0.236758, acc 0.92
2018-04-17T05:50:16.683789: step 1678, loss 0.177044, acc 0.94
2018-04-17T05:50:16.704900: step 1679, loss 0.131467, acc 0.93
2018-04-17T05:50:16.730048: step 1680, loss 0.160117, acc 0.93

Evaluation:
2018-04-17T05:50:16.830729: step 1680, loss 0.48709, acc 0.793696, rec 0.741163, pre 0.83014, f1 0.783133

2018-04-17T05:50:16.853015: step 1681, loss 0.139979, acc 0.94
2018-04-17T05:50:16.873223: step 1682, loss 0.196365, acc 0.94
2018-04-17T05:50:16.889551: step 1683, loss 0.242206, acc

2018-04-17T05:50:19.723138: step 1790, loss 0.149446, acc 0.96
2018-04-17T05:50:19.740377: step 1791, loss 0.104057, acc 0.972603
2018-04-17T05:50:19.761937: step 1792, loss 0.103804, acc 0.98
2018-04-17T05:50:19.783580: step 1793, loss 0.18267, acc 0.95
2018-04-17T05:50:19.805058: step 1794, loss 0.131327, acc 0.94
2018-04-17T05:50:19.826063: step 1795, loss 0.150207, acc 0.96
2018-04-17T05:50:19.847182: step 1796, loss 0.145351, acc 0.96
2018-04-17T05:50:19.868757: step 1797, loss 0.184593, acc 0.91
2018-04-17T05:50:19.890488: step 1798, loss 0.170001, acc 0.94
2018-04-17T05:50:19.912361: step 1799, loss 0.132853, acc 0.98
2018-04-17T05:50:19.935789: step 1800, loss 0.132393, acc 0.945205

Evaluation:
2018-04-17T05:50:20.038913: step 1800, loss 0.775161, acc 0.676791, rec 0.942987, pre 0.616704, f1 0.745717

Saved model checkpoint to /home/saulgrimaldo1/sarcasm_detection_w266/runs/1523944169/checkpoints/model-1800


Test Set:
2018-04-17T05:50:20.134301: step 1800, loss 0.703469, acc 